In [1]:
import pandas as pd
from raug.loader import get_data_loader
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)

In [2]:
_csv_path_train = "/home/a52550/Desktop/datasets/ISIC2017/train/ISIC-2017_Training_Part3_GroundTruth.csv"
_imgs_folder_train = "/home/a52550/Desktop/datasets/ISIC2017/train/ISIC-2017_Training_Data/"
_imgs_folder_train_cropped = "/home/a52550/Desktop/datasets/ISIC2017/train/cropped_images/"

_csv_path_validation = "/home/a52550/Desktop/datasets/ISIC2017/validation/ISIC-2017_Validation_Part3_GroundTruth.csv"
_imgs_folder_validation = "/home/a52550/Desktop/datasets/ISIC2017/validation/ISIC-2017_Validation_Data/"
_imgs_folder_validation_cropped = "/home/a52550/Desktop/datasets/ISIC2017/validation/cropped_images/"
_csv_path_test = "/home/a52550/Desktop/datasets/ISIC2017/test/ISIC-2017_Test_v2_Part3_GroundTruth.csv"
_imgs_folder_test = "/home/a52550/Desktop/datasets/ISIC2017/test/ISIC-2017_Test_Data/"
_imgs_folder_test_cropped = "/home/a52550/Desktop/datasets/ISIC2017/test/cropped_images/"

_csv_path_all_metrics = "results/all_metrics.csv"


In [3]:
train_data = pd.read_csv(_csv_path_train)
validation_data = pd.read_csv(_csv_path_validation)
test_data = pd.read_csv(_csv_path_test)
test_data.head()

,image_id,category
0,ISIC_0012086,2
1,ISIC_0012092,0
2,ISIC_0012095,0
3,ISIC_0012134,2
4,ISIC_0012136,2


In [4]:
train_data.category.value_counts()

0    1372
1     374
2     254
Name: category, dtype: int64

In [5]:
train_imgs_path = ["{}{}.jpg".format(_imgs_folder_train, img_id) for img_id in train_data.image_id.values]

loader = get_data_loader(train_imgs_path, train_data.category.values, batch_size=10)


In [6]:
def visualise_dataloader(dl, id_to_label=None, with_outputs=True):
    total_num_images = len(dl.dataset)
    idxs_seen = []
    class_0_batch_counts = []
    class_1_batch_counts = []

    for i, batch in enumerate(dl, 0):

        idxs = batch[0][:, 0].tolist()
        classes = batch[0][:, 1]
        class_ids, class_counts = classes.unique(return_counts=True)
        class_ids = set(class_ids.tolist())
        class_counts = class_counts.tolist()

        idxs_seen.extend(idxs)

        if len(class_ids) == 2:
            class_0_batch_counts.append(class_counts[0])
            class_1_batch_counts.append(class_counts[1])
        elif len(class_ids) == 1 and 0 in class_ids:
            class_0_batch_counts.append(class_counts[0])
            class_1_batch_counts.append(0)
        elif len(class_ids) == 1 and 1 in class_ids:
            class_0_batch_counts.append(0)
            class_1_batch_counts.append(class_counts[0])
        else:
            raise ValueError("More than two classes detected")

    if with_outputs:
        fig, ax = plt.subplots(1, figsize=(15, 15))

        ind = np.arange(len(class_0_batch_counts))
        width = 0.35

        ax.bar(
            ind,
            class_0_batch_counts,
            width,
            label=(id_to_label[0] if id_to_label is not None else "0"),
        )
        ax.bar(
            ind + width,
            class_1_batch_counts,
            width,
            label=(id_to_label[1] if id_to_label is not None else "1"),
        )
        ax.set_xticks(ind, ind + 1)
        ax.set_xlabel("Batch index", fontsize=12)
        ax.set_ylabel("No. of images in batch", fontsize=12)
        ax.set_aspect("equal")

        plt.legend()
        plt.show()

        num_images_seen = len(idxs_seen)

        print(
            f'Avg Proportion of {(id_to_label[0] if id_to_label is not None else "Class 0")} per batch: {(np.array(class_0_batch_counts) / 10).mean()}'
        )
        print(
            f'Avg Proportion of {(id_to_label[1] if id_to_label is not None else "Class 1")} per batch: {(np.array(class_1_batch_counts) / 10).mean()}'
        )
        print("=============")
        print(f"Num. unique images seen: {len(set(idxs_seen))}/{total_num_images}")
    return class_0_batch_counts, class_1_batch_counts, idxs_seen

visualise_dataloader(loader)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/a52550/Desktop/masters/cnn/learning_cycle/benchmarks/isic/venv/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/a52550/Desktop/masters/cnn/learning_cycle/benchmarks/isic/venv/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 61, in fetch
    return self.collate_fn(data)
  File "/home/a52550/Desktop/masters/cnn/learning_cycle/benchmarks/isic/venv/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/home/a52550/Desktop/masters/cnn/learning_cycle/benchmarks/isic/venv/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 143, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards c